In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report




In [30]:
data = pd.read_excel('./data/data.xlsx')
data = data.iloc[:, :2]
data

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포
...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오
38590,재미가 없으니 망하지,혐오
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오


In [31]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data['Sentence'], data['Emotion'], test_size=0.2, random_state=42)


In [32]:
# 특성 추출
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [33]:
# 모델 학습
class MultinomialNB:
    def __init__(self, alpha=1.0):
        self.alpha = alpha
        
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.class_count = np.zeros(len(self.classes))
        self.feature_count = np.zeros((len(self.classes), X.shape[1]))
        
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.class_count[i] = X_c.shape[0]
            self.feature_count[i, :] = X_c.sum(axis=0)
        
        self.class_log_prior_ = np.log(self.class_count / self.class_count.sum())
        self.feature_log_prob_ = np.log((self.feature_count + self.alpha) / 
                                        (self.feature_count.sum(axis=1)[:, np.newaxis] + self.alpha * X.shape[1]))
    
    def predict_log_proba(self, X):
        return X @ self.feature_log_prob_.T + self.class_log_prior_
    
    def predict(self, X):
        return self.classes[np.argmax(self.predict_log_proba(X), axis=1)]

# 모델
model = MultinomialNB()

In [34]:
#모델학습
model.fit(X_train_vectorized, y_train)

In [35]:
# 예측
y_pred = model.predict(X_test_vectorized)

In [36]:
# 결과 평가
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          공포       0.42      0.38      0.40      1096
          놀람       0.33      0.28      0.30      1177
          분노       0.34      0.33      0.33      1150
          슬픔       0.44      0.42      0.43      1020
          중립       0.32      0.15      0.20       974
          행복       0.38      0.74      0.50      1172
          혐오       0.29      0.24      0.26      1130

    accuracy                           0.37      7719
   macro avg       0.36      0.36      0.35      7719
weighted avg       0.36      0.37      0.35      7719



In [37]:
# 새로운 문장 분류 예시
new_sentences = [
    "이 영화는 정말 무서웠어요",
    "오늘 날씨가 좋아서 기분이 좋아요",
    "갑자기 큰 소리가 나서 놀랐어요"
]
new_sentences_vectorized = vectorizer.transform(new_sentences)
predictions = model.predict(new_sentences_vectorized)

for sentence, emotion in zip(new_sentences, predictions):
    print(f"문장: {sentence}")
    print(f"예측된 감정: {emotion}\n")

문장: 이 영화는 정말 무서웠어요
예측된 감정: 행복

문장: 오늘 날씨가 좋아서 기분이 좋아요
예측된 감정: 행복

문장: 갑자기 큰 소리가 나서 놀랐어요
예측된 감정: 공포

